## Ollama - DeepSeek-Coder-V2

In this notebook, we show an example of using DeepSeek-Coder-V2 to solve a coding problem, that was copied over from the HumanEval dataset.

In [1]:
from not_again_ai.local_llm.chat_completion import chat_completion
from not_again_ai.local_llm.ollama.ollama_client import ollama_client
from not_again_ai.local_llm.prompts import chat_prompt
from not_again_ai.local_llm.tokens import load_tokenizer, num_tokens_from_messages

client = ollama_client()

MODEL_NAME = "deepseek-coder-v2:16b"

In [2]:
FUNCTION_HEADER = '''\
def correct_bracketing(brackets: str):
    """brackets is a string of "(" and ")". return True if every opening bracket has a corresponding closing bracket. 
    >>> correct_bracketing("(") False >>> correct_bracketing("()") True 
    >>> correct_bracketing("(()())") True 
    >>> correct_bracketing(")(()") False"""'''


messages_unformatted = [
    {
        "role": "system",
        "content": """You are writing high-quality and accurate Python code. 
You have a function header with a docstring and your goal is to implement the function such that it passes the test cases given in the docstring.
Denote your code with triple backticks (```) and write the code inside the code block. Include the function header and the function implementation.""",
    },
    {
        "role": "user",
        "content": """{{FUNCTION_HEADER}}

# Implement the function""",
    },
]

messages = chat_prompt(messages_unformatted, {"FUNCTION_HEADER": FUNCTION_HEADER})

# Count the number of tokens in the messages
tokenizer = load_tokenizer(model=MODEL_NAME, provider="ollama")
num_tokens = num_tokens_from_messages(messages, tokenizer=tokenizer, provider="ollama")
print(f"Number of tokens from messages: {num_tokens}")

Number of tokens from messages: 172


In [3]:
response = chat_completion(messages, model=MODEL_NAME, client=client)
print(f'GENERATED RESPONSE:\n{response["message"]}')

GENERATED RESPONSE:
 ```python
def correct_bracketing(brackets: str):
    """brackets is a string of "(" and ")". Return True if every opening bracket has a corresponding closing bracket.
    
    >>> correct_bracketing("(") 
    False 
    >>> correct_bracketing("()") 
    True 
    >>> correct_bracketing("(()())") 
    True 
    >>> correct_bracketing(")(()") 
    False
    """
    stack = []
    for bracket in brackets:
        if bracket == '(':
            stack.append(bracket)
        elif bracket == ')':
            if not stack:
                return False
            stack.pop()
    return len(stack) == 0
```
